<a href="https://colab.research.google.com/github/JollygreenG-10/Cybersecurity/blob/main/Federated_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install --quiet tensorflow-federated
!pip install --quiet --upgrade dp-accounting
import numpy as np
import random
import cv2
import pandas as pd
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
import collections
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import tensorflow_federated as tff


In [4]:
data_full = pd.read_csv('Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
data_full = pd.DataFrame(data_full)
data_full.columns = data_full.columns.str.replace(" ", "")

In [ ]:
data_full.info()
data_full['Label'].value_counts()
data_full.describe()

In [5]:
mapping_dict = {'BENIGN':0, 'DDoS':1}
data_full['Label'] = data_full['Label'].map(mapping_dict)
data_full = data_full.sample(frac = 1)
labels = data_full['Label']
data_full.drop(['Label', 'FlowBytes/s', 'FlowPackets/s'], axis = 1, inplace = True)

for col in data_full.columns:
  scaler = MinMaxScaler()
  try:
    data_full[col] = scaler.fit_transform(data_full[[col]])
  except:
    print(col)


x_train, x_test, y_train, y_test = train_test_split(data_full, labels, test_size = 0.2, random_state = 10)


In [73]:
def build(shape):
    model = Sequential()
    model.add(Dense(shape, input_shape=(shape,)))
    model.add(Activation("relu"))
    model.add(Dense(50))
    model.add(Activation("relu"))
    model.add(Dense(50))
    model.add(Activation("relu"))
    model.add(Dense(1, activation = 'sigmoid'))
    return model

lr = 0.0001
comms_round = 100
loss=tf.keras.losses.BinaryCrossentropy()
metrics = ['accuracy', 'Precision', 'Recall']

In [ ]:
x_train = np.array(x_train).astype('float')
y_train = np.array(y_train)
SGD_model = build(x_train.shape[1])

SGD_model.compile(loss=loss,
              optimizer=Adam(),
              metrics=metrics)

# fit the SGD training data to model
fitted_model = SGD_model.fit(x_train, y_train, epochs=50, batch_size=100)

# Federated Learning Model 1


##Create nodes for separation of data

In [23]:
class MLP:
    @staticmethod
    def build(shape):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(1, activation = 'sigmoid'))
        return model

lr = 0.0001
comms_round = 1
loss=tf.keras.losses.BinaryCrossentropy()
metrics = ['accuracy', 'Precision', 'Recall']

In [7]:
def create_nodes(data, labels, num_nodes: int):
  init = 'Node'
  node_names = [init+str(i+1) for i in range(num_nodes)]
  data = np.array(data)

  data = list(zip(data, labels))
  random.shuffle(data)

  shard_size = len(data)//num_nodes

  shards = [data[i:i+shard_size] for i in range(0, shard_size*num_nodes, shard_size)]
  assert(len(shards) == num_nodes)

  return {node_names[i] : shards[i] for i in range(len(node_names))}

nodes = create_nodes(x_train, y_train, 50)

def batch_data(shard):
  data, label = zip(*shard)
  dataset = tf.data.Dataset.from_tensor_slices((np.array(data), np.array(label)))
  return dataset.shuffle(len(label)).batch(32)



In [8]:
#process and batch the training data for each client
nodes_batched = dict()
for (node_name, data) in nodes.items():

  nodes_batched[node_name] = batch_data(data)

#process and batch the test set
test_batched = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(len(y_test))

In [37]:
from numpy.random import normal
def weight_scaling_factor(node_train, node_name):
    node_names = list(node_train.keys())
    #get the bs
    bs = list(node_train[node_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(node_train[node_name]).numpy() for node_name in node_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(node_train[node_name]).numpy()*bs
    return local_count/global_count



def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final


def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    average_weights = list()
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        average_weights.append(layer_mean)
    return average_weights

def test_model(x_test, y_test,  model):
    pred = model.predict(x_test).round(1).astype(int)
    print(pred)
    acc = accuracy_score(y_test, pred)
    prec = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print(f'Global Model Accuracy is {acc}, precision is {prec}, recall is {recall}, f1 is {f1}')
    return

In [25]:

#initialize global model
smlp_global = MLP()
global_model = smlp_global.build(76)

#commence global training loop
for comm_round in range(1):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    #initial list to collect local model weights after scaling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    node_names= list(nodes_batched.keys())
    random.shuffle(node_names)

    #loop through each client and create new local model
    for node in node_names:
        smlp_local = MLP()
        local_model = smlp_local.build(76)
        local_model.compile(loss=loss,
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=metrics)

        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        #fit local model with client's data
        local_model.fit(nodes_batched[node], epochs=3)

        #scale the model weights and add to list
        scaling_factor = weight_scaling_factor(nodes_batched, node)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        #clear session to free memory after each communication round
        K.clear_session()

    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)

    #update global model
    global_model.set_weights(average_weights)

Epoch 1/3
113/113 [==============================] - 1s 3ms/step - loss: 0.1691 - accuracy: 0.9568 - precision: 0.9413 - recall: 0.9842
Epoch 2/3
113/113 [==============================] - 0s 3ms/step - loss: 0.0351 - accuracy: 0.9875 - precision: 0.9801 - recall: 0.9980
Epoch 3/3
113/113 [==============================] - 0s 3ms/step - loss: 0.0279 - accuracy: 0.9900 - precision: 0.9844 - recall: 0.9980
Epoch 1/3
113/113 [==============================] - 1s 3ms/step - loss: 0.1734 - accuracy: 0.9579 - precision: 0.9415 - recall: 0.9868
Epoch 2/3
113/113 [==============================] - 0s 3ms/step - loss: 0.0385 - accuracy: 0.9856 - precision: 0.9765 - recall: 0.9985
Epoch 3/3
113/113 [==============================] - 0s 3ms/step - loss: 0.0324 - accuracy: 0.9884 - precision: 0.9817 - recall: 0.9980
Epoch 1/3
113/113 [==============================] - 1s 4ms/step - loss: 0.1697 - accuracy: 0.9568 - precision: 0.9436 - recall: 0.9830
Epoch 2/3
113/113 [=============================

In [ ]:
test_model(x_test, y_test, global_model)

#Federated Learning with Differential Privacy

In [ ]:
def add_noise(modelweights):
  for layer in modelweights:
    noise = normal(loc=0.0, scale=1.0, size=layer.shape)
    layer += noise
  return modelweights

In [ ]:
#initialize global model
smlp_global = MLP()
global_model = smlp_global.build(76)

#commence global training loop
for comm_round in range(comms_round):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    #initial list to collect local model weights after scaling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    node_names= list(nodes_batched.keys())
    random.shuffle(node_names)

    #loop through each client and create new local model
    for node in node_names:
        smlp_local = MLP()
        local_model = smlp_local.build(76)
        local_model.compile(loss=loss,
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=metrics)

        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        #fit local model with client's data
        local_model.fit(nodes_batched[node], epochs=3)

        #scale the model weights and add to list
        scaling_factor = weight_scaling_factor(nodes_batched, node)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_weights= add_noise(scaled_weights)
        scaled_local_weight_list.append(scaled_weights)



        #clear session to free memory after each communication round
        K.clear_session()

    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)

    #update global model
    global_model.set_weights(average_weights)